In [ ]:
%pip install openai langchain PyPDF2 pypdf
%pip install -U langchain-community
%pip install python-dotenv

In [170]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

## PDF 바탕으로 1차로 전처리 후 md로 저장. 
## 그 md 파일을 바탕으로 답변 생성

(정확도 증가)

### Full text를 다 넣어줬을 때 (카테고라이징 전)

In [443]:
question = "프로젝트 코드 설정 조건에 대해 알려주세요."

with open("document/example.md", 'r', encoding='utf-8') as txt_file:
        pdf_text = txt_file.read()

prompt = f"""
# 목표
- 사용자 질문을 분석하여 핵심 정보를 파악합니다.
- 제공된 문서에서 관련 내용을 찾아 정확하고 구체적인 답변을 제공합니다.
- 문서에 해당 내용이 없을 경우, "제공된 문서에는 해당 정보가 없습니다."라고 응답하세요.
- 전문 용어가 필요할 경우, 제공된 용어집을 참고하여 적절한 용어를 사용하세요.
- 응답 형식은 간결하지만, 필요하면 세부적인 정보를 포함해야 합니다.

# 제약 사항
- 문서에 있는 정보만 사용하여 답변하세요.
- 불확실한 정보는 생성하지 마세요.
- 질문과 직접적으로 관련된 문서 내용을 인용하여 답변을 작성하세요.

# 입력 문서
다음은 사용자가 제공한 문서 내용입니다.
{pdf_text}

# 사용자의 질문 
{question}

"""

response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "문서 내용을 참고해서 질문에 대해 답변해주세요."},
            {"role": "user", "content": prompt}
        ]
    )

print("답변: " + response.choices[0].message.content)

답변: 제공된 문서에서 "프로젝트 코드"에 대한 정보는 제 2 절 시스템관리의 "프로젝트 정보등록" 부분에 언급되어 있습니다. 

1) **프로젝트 기본정보 등록**
   - 프로젝트번호: 20 자리 이내로 등록 가능하며, 계약일을 입력해야 합니다.
   - 거래처 및 사업장을 연결하여 등록해야 합니다.

프로젝트 코드에 대한 더 구체적인 설정 조건은 문서에서 찾을 수 없습니다. 제공된 문서에는 프로젝트 관련 내용이 간략하게 언급되어 있으나, 적절한 상세 설명은 없으므로, 추가적인 정보는 관련 시스템의 매뉴얼이나 지원팀에 문의해야 할 것입니다.


### 카테고라이징 후

문서 전처리

In [173]:
import re

# 카테고리 생성 함수
def generate_categorized_txt(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    prompt = f"""
    # 목표
    - 문서의 주요 내용을 파악하여 논리적인 목차를 구성합니다.
    - 목차는 계층적으로 구성되며, 최대 2단계까지 세분화됩니다.
    - 일관된 포맷을 유지해야 합니다.

    # 출력 형식
    출력은 다음과 같은 형식을 따라야 합니다.
    1. [대주제] 1.1 [중주제]

    # 지침
    - 제목을 최대한 간결하게 작성합니다.
    - 중복되는 내용을 배제하고, 문서의 핵심 내용을 반영합니다.
    - 문서의 흐름을 고려하여 논리적인 순서로 정리합니다.

    # 입력
    다음은 분석할 Markdown 문서입니다:

    {content}

    # 출력 예시
    1. 원가계산체계정의
        1.1 원가계산기준
        1.2 원가구조 및 원가계정체계
        1.3 표준재고단가등록
        1.4 제품재투입단가등록/제품이동단가등록

    # 출력
    (여기에 목차를 출력)
    ...
    """

    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[ 
                {"role": "system", "content": "당신은 문서 분석을 전문으로 하는 AI입니다. 주어진 Markdown 파일의 내용을 기반으로 정확하고 논리적인 목차를 생성하세요."},
                {"role": "user", "content": prompt}
            ],
            temperature = 0.2
        )

    table_of_contents = response.choices[0].message.content

    output_file_path = "document/categorized.md"
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(table_of_contents)

    return table_of_contents

In [ ]:
input_file_path = "document/example.md"
generate_categorized_txt(input_file_path)

In [ ]:
# 파일의 내용을 불러오는 함수
def load_data_from_file(filename):
    texts = []
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            text = line.strip()
            if text:
                texts.append(text)
    return texts

In [ ]:
categories = load_data_from_file("document/categorized.md")

def select_tag_from_category(categories):
    prompt = f"""

    # 목표
    - 목차의 최상위 레벨(1단계) 항목을 태그로 변환합니다.
    - 태그는 **반드시** 최상위 항목의 제목을 그대로 사용해야 합니다.
    - 태그는 str 형식으로 반환되어야 합니다.

    # 입력
    다음은 목차입니다:
    {categories}

    # 출력 형식
    - 태그는 반드시 아래 형식을 따라야 합니다.
    ["1. 최상위 항목1", "2. 최상위 항목2", "3. 최상위 항목3"]

    # 예시
    ## 입력
    카테고리:
    1. 대학교
        1.1 입학 준비
        1.2 학과 소개
    2. 중학교
        2.1 학생 지원
        2.2 교육과정
    3. 교육정책
        3.1 교육과정 개편
        3.2 교육과정 평가
    4. 입시제도
        4.1 입시제도 개편
        4.2 입시제도 평가
    
    ## 출력
    ["1. 대학교", "2. 중학교", "3. 교육정책", "4. 입시제도"]

    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 문서 분석을 전문으로 하는 AI입니다. 주어진 목차를 분석하여 최상위 항목을 기반으로 태그를 생성해야 합니다."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

tags = select_tag_from_category(categories)
print(tags)

["1. ERP-IU 소개", "2. 시스템 관리", "3. 회계기준정보", "4. 재무관리", "5. 건별 반제관리", "6. 외화관리", "7. 재무제표 관리", "8. 결산관리", "9. 손익관리", "10. 원가관리", "11. 배부관리"]


### 사용자 질문에서 관련 카테고리 추출

In [361]:
# 사용자가 질문을 입력하면 해당 카테고리 항목을 반환하는 함수
def score_tag_for_question(question):

    prompt = f"""
    # 목표
    - 사용자의 질문을 분석하여 핵심 주제를 파악합니다.
    - 주어진 카테고리를 바탕으로 모든 태그에 대해 0~100 사이로 점수를 매깁니다. 
    - 출력 형식을 반드시 지켜야 하며, 총점만 출력하면 됩니다.

    # 입력
    사용자의 질문과 카테고리 및 태그 목록이 주어집니다.

    질문: {question}
    
    카테고리: {categories}
    태그: {tags}

    # 평가 기준
    - 관련성 (40점): 사용자의 질문과 가장 관련성이 높은 태그를 선택했는가?
    - 의미적 유사도 (30점): 질문의 의도가 해당 태그와 유사한가?
    - 논리적 적절성 (30점): 태그가 질문의 문맥과 논리적으로 일관되는가?
    
    # 출력 형식
    태그명1: 85
    태그명2: 60
    태그명3: 95

    # 지침
    - 반드시 모든 태그에 대해 점수를 매겨야합니다.

    # 예시
    ## 입력
    질문: "대학교 입학 준비는 어떻게 하면 좋을까요?"

    카테고리:
    1. 대학교
        1.1 입학 준비
        1.2 학과 소개
    2. 중학교
        2.1 학생 지원
        2.2 교육과정
    3. 교육정책
        3.1 교육과정 개편
        3.2 교육과정 평가
    4. 입시제도
        4.1 입시제도 개편
        4.2 입시제도 평가
    태그: ["1. 대학교", "2. 중학교", "3. 교육정책", "4. 입시제도"]

    ## 출력
    1. 대학교: 85
    2. 중학교: 50
    3. 교육정책: 75
    4. 입시제도: 90

    """
    
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 사용자의 질문과 미리 정의된 카테고리를 비교하여 가장 적절한 태그를 선택하는 AI입니다. 반드시 제공된 태그 중에서만 선택해야 합니다."},
                {"role": "user", "content": prompt}
            ],
            temperature = 0.2
        )
    
    response_text = response.choices[0].message.content

    # 모델의 응답을 기반으로 해당 카테고리 항목을 반환
    return response_text

In [412]:
# 80 이상 점수를 받은 카테고리 항목을 반환하는 함수
def filter_high_scores(text: str, threshold: int = 70):
    high_scores = []
    
    matches = re.findall(r"(.+?): (\d+)", text)
    
    for subject, score in matches:
        if int(score) >= threshold:
            high_scores.append((subject, int(score)))
    
    high_scores.sort(key=lambda x: x[1], reverse=True)
    
    return [subject for subject, score in high_scores]


In [ ]:
import json, ast

# JSON 파일을 읽어와서 선택된 태그에 해당하는 텍스트를 반환하는 함수
def get_text_by_tags(json_file: str, selected_tags: list):

    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    selected_texts = {tag: data[tag] for tag in selected_tags if tag in data}

    return selected_texts

json_file_path = "document/splitted.json"

### 최종 답변 생성

용어집 참고 전

In [193]:
# 최종 답변 생성 (용어집 참고 X)
def generate_answer_without_glossary(question, input_text):
    prompt = f"""
    # 목표
    - 사용자 질문을 분석하여 핵심 정보를 파악합니다.
    - 제공된 문서에서 관련 내용을 찾아 정확한 답변을 제공합니다.
    - 문서에 해당 내용이 없을 경우, "제공된 문서에는 해당 정보가 없습니다."라고 응답하세요.
    - 응답 형식은 간결하지만, 필요하면 세부적인 정보를 포함해야 합니다.

    # 제약 사항
    - 문서에 있는 정보만 사용하여 답변하세요.
    - 불확실한 정보는 생성하지 마세요.
    - 질문과 직접적으로 관련된 문서 내용을 인용하여 답변을 작성하세요.

    # 입력 문서
    다음은 사용자가 제공한 문서 내용입니다.
    {input_text}

    # 사용자의 질문 
    {question}
    
    """
    
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 사용자가 제공한 문서를 기반으로 질문에 답하는 AI 비서입니다. 답변을 작성할 때 반드시 주어진 문서의 내용을 바탕으로 하며, 문서에 없는 정보는 절대 생성하지 마십시오."},
                {"role": "user", "content": prompt}
            ],
            temperature = 0.2
        )
    
    response_text = response.choices[0].message.content
    print(response_text)

    # 모델의 응답을 기반으로 해당 카테고리 항목을 반환
    return response_text

용어집 참고 후

In [196]:
# 용어집 로딩
def load_glossary(glossary_file: str):

    with open(glossary_file, "r", encoding="utf-8") as f:
        glossary = json.load(f)
    return glossary

glossary_file = "glossary.json"
glossary = load_glossary(glossary_file)

In [262]:
#  최종 답변 생성 (용어집 참고 O) - 정확도 높음/ hallucination 낮음/ 답변 생성 불가능한 경우 발생
def generate_answer_with_glossary_accurate(question, input_text, glossary):
    prompt = f"""
    # 목표
    - 사용자 질문을 분석하여 핵심 정보를 파악합니다.
    - 제공된 문서에서 관련 내용을 찾아 정확하고 구체적인 답변을 제공합니다.
    - 문서에 해당 내용이 없을 경우, "제공된 문서에는 해당 정보가 없습니다."라고 응답하세요.
    - 전문 용어가 필요할 경우, 제공된 용어집을 참고하여 적절한 용어를 사용하세요.
    - 응답 형식은 간결하지만, 필요하면 세부적인 정보를 포함해야 합니다.

    # 제약 사항
    - 문서에 있는 정보를 사용하여 답변하세요. 
    - 불확실한 정보는 생성하지 마세요.
    - 질문과 직접적으로 관련된 문서 내용을 인용하여 답변을 작성하세요.

    # 용어집
    아래는 전문 용어의 정의입니다.  
    필요할 경우 적절한 용어를 사용하여 답변을 작성하세요.
    {glossary}

    # 입력 문서
    다음은 사용자가 제공한 문서 내용입니다.
    {input_text}

    # 사용자의 질문 
    {question}
    
    """
    
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 사용자가 제공한 문서를 기반으로 질문에 답하는 AI 비서입니다. 답변을 작성할 때 반드시 주어진 문서의 내용을 바탕으로 생성하세요. 또한, 전문 용어가 필요할 경우 제공된 용어집을 참고하여 정확한 용어를 사용하십시오"},
                {"role": "user", "content": prompt}
            ],
            temperature = 0.2
        )
    
    response_text = response.choices[0].message.content
    print(response_text)

    # 모델의 응답을 기반으로 해당 카테고리 항목을 반환
    return response_text

In [258]:
# 최종 답변 생성 (용어집 참고 O) - 정확도 낮음/ hallucination 높음/ 답변 생성 불가능한 경우 거의 없음
def generate_answer_with_glossary_hallucination(question, input_text, glossary):
    prompt = f"""
    # 목표
    - 사용자 질문을 분석하여 핵심 정보를 파악합니다.
    - 제공된 문서에서 관련 내용을 찾아 정확하고 구체적인 답변을 제공합니다.
    - 전문 용어가 필요할 경우, 제공된 용어집을 참고하여 적절한 용어를 사용하세요.
    - 응답 형식은 간결하지만, 필요하면 세부적인 정보를 포함해야 합니다.

    # 제약 사항
    - 문서에 있는 정보를 사용하여 답변하세요. 

    # 용어집
    아래는 전문 용어의 정의입니다.  
    필요할 경우 적절한 용어를 사용하여 답변을 작성하세요.
    {glossary}

    # 입력 문서
    다음은 사용자가 제공한 문서 내용입니다.
    {input_text}

    # 사용자의 질문 
    {question}
    
    """
    
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 사용자가 제공한 문서를 기반으로 질문에 답하는 AI 비서입니다. 답변을 작성할 때 반드시 주어진 문서의 내용을 바탕으로 생성하세요. 또한, 전문 용어가 필요할 경우 제공된 용어집을 참고하여 정확한 용어를 사용하십시오"},
                {"role": "user", "content": prompt}
            ],
            temperature = 0.5
        )
    
    response_text = response.choices[0].message.content
    print(response_text)

    # 모델의 응답을 기반으로 해당 카테고리 항목을 반환
    return response_text

이미지 추가

In [ ]:
pip install matplotlib


In [143]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Matplotlib is building the font cache; this may take a moment.


In [444]:
png_images = load_data_from_file("image_tag.md")

# 사용자가 질문을 입력하면 해당 카테고리 항목을 반환하는 함수
def select_image(question):

    prompt = f"""
    
    # 목표
    - 사용자가 제공한 질문을 이해하고 핵심 내용을 파악합니다.
    - 이미지 리스트에서 질문과 가장 연관성이 높은 이미지를 찾습니다.
    - 선택한 이미지의 태그를 반환합니다.
    - 적절한 이미지가 없을 경우, "None"을 반환합니다.

    # 이미지 태그 예시
    ["1. 이미지1", "2. 이미지2", "3. 이미지3", "4. 이미지4", "5. 이미지5"]

    # 출력 형식
    - 관련 이미지가 있을 경우: 
    ["3. 이미지3", "5. 이미지5"]
    - 관련 이미지가 없을 경우: 
    ["None"]

    # 사용자 질문
    {question}

    # 이미지 태그
    {png_images}

    """
    
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "당신은 이미지 데이터베이스에서 사용자의 질문과 가장 관련성이 높은 이미지를 찾는 AI 비서입니다. 당신의 목표는 질문에 가장 적합한 이미지를 식별하고 해당 이미지의 태그를 반환하는 것입니다."},
                {"role": "user", "content": prompt}
            ]
        )
    
    response_text = response.choices[0].message.content
    print(response_text)

    # 모델의 응답을 기반으로 해당 카테고리 항목을 반환
    return response_text


In [187]:
def display_images(image_titles):
    image_filenames = [f"image/{title}.png" for title in image_titles]

    for image_path in image_filenames:
        if os.path.exists(image_path):
            img = mpimg.imread(image_path)
            plt.imshow(img)
            plt.axis('off')
            plt.show()
        else:
            print(f"Error: Image file '{image_path}' not found.")

# 여기부터 실행

In [439]:
# 예시 질문 입력
question = "프로젝트 코드 설정 조건에 대해 알려주세요."

# 각 태그의 점수 반환
tag_score = score_tag_for_question(question)
print(tag_score)

1. ERP-IU 소개: 70
2. 시스템 관리: 90
3. 회계기준정보: 60
4. 재무관리: 50
5. 건별 반제관리: 40
6. 외화관리: 30
7. 재무제표 관리: 50
8. 결산관리: 40
9. 손익관리: 60
10. 원가관리: 30
11. 배부관리: 20


In [445]:
# 80 이상 점수를 받은 태그 항목을 반환
category_answer = filter_high_scores(tag_score)

print(category_answer)

['2. 시스템 관리', '1. ERP-IU 소개']


In [ ]:
# 선택된 태그의 실제 본문 반환
extracted_texts = get_text_by_tags(json_file_path, category_answer)
    
for tag, text in extracted_texts.items():
    print(f"### {tag} ###\n{text}\n")

In [447]:
# 용어집 참고 X 답변
final_answer_without_glossary = generate_answer_without_glossary(question, extracted_texts)

제공된 문서에는 "프로젝트 코드 설정 조건"에 대한 구체적인 정보가 없습니다.


In [448]:
# 용어집 참고 O 답변 (accuracy 높음, but 답변 생성 불가)
final_answer_with_glossary = generate_answer_with_glossary_accurate(question, extracted_texts, glossary)

제공된 문서에는 프로젝트 코드 설정 조건에 대한 구체적인 내용이 없습니다.


In [449]:
# 용어집 참고 O 답변 (hallucination 높음, but 답변 생성 가능)
final_answer_with_glossary = generate_answer_with_glossary_hallucination(question, extracted_texts, glossary)

프로젝트 코드 설정 조건에 대한 정보는 다음과 같습니다:

- **프로젝트번호**: 20자리 이내로 등록 가능합니다.
- **계약일**: 프로젝트의 계약일을 입력해야 하며, 이는 필수 선택사항입니다.
- **거래처**: 해당 프로젝트에 연결할 거래처를 지정해야 합니다.
- **사업장**: 프로젝트를 진행하는 사업장을 연결해야 합니다.

이와 같은 조건을 충족하여 프로젝트 코드를 설정하면 됩니다.


In [450]:
image = select_image(question)
image = ast.literal_eval(image)

display_images(image)

["None"]
Error: Image file 'image/None.png' not found.


# 결론

### 프롬프트 구체적으로 작성하면 성능 좋다